<a href="https://colab.research.google.com/github/MK316/Myapps/blob/main/mrkim21apps/korean_english.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loanword pronunciation practice application

# Install packages

In [ ]:
!pip install gtts gradio

In [ ]:
!pip install pydub

# Data to read

FWords_phonemes.csv

In [ ]:
import pandas as pd

url = "https://raw.githubusercontent.com/MK316/Spring2024/main/data/FKE.csv"
# Read the CSV file
df = pd.read_csv(url, encoding='utf-8')

Selecting data

In [ ]:
df.head()

# Gradio

Remove unnecessary strings (number) for example.

```
df['Entry'] = df['Entry'].str.replace(r"\(\d+\)", "", regex=True)
```

Gradio implementation

In [ ]:
#@markdown [1] Gradio implemented app (audo generate without Submit)
import gradio as gr
from gtts import gTTS
from pydub import AudioSegment
import pandas as pd
import tempfile
from io import BytesIO

# Assuming 'df' is your DataFrame and is already loaded at this point

# Function to generate speech and combine two audios
def generate_speech(index):
    # Adjust for zero-based index
    adjusted_index = index - 1

    # Check if the provided index is within the range of the DataFrame
    if adjusted_index < 0 or adjusted_index >= len(df):
        return "Index out of range", None

    # Get the Korean and English words from the DataFrame
    korean_word = df.iloc[adjusted_index]['Entry']
    english_word = df.iloc[adjusted_index]['Eword']

    # Generate TTS for Korean
    tts_ko = gTTS(text=korean_word, lang='ko')
    ko_buffer = BytesIO()
    tts_ko.write_to_fp(ko_buffer)
    ko_buffer.seek(0)
    ko_audio = AudioSegment.from_file(ko_buffer, format="mp3")

    # Generate TTS for English
    tts_en = gTTS(text=english_word, lang='en')
    en_buffer = BytesIO()
    tts_en.write_to_fp(en_buffer)
    en_buffer.seek(0)
    en_audio = AudioSegment.from_file(en_buffer, format="mp3")

    # Add a 2-second pause between the words
    pause = AudioSegment.silent(duration=1000)

    # Combine the two audios with a pause in between
    combined_audio = ko_audio + pause + en_audio

    # Save combined audio to a temporary file
    with tempfile.NamedTemporaryFile(delete=False, suffix='.mp3') as fp:
        combined_audio.export(fp, format='mp3')
        # Return the file path
        return fp.name

# Use the Blocks API to create a more custom layout
with gr.Blocks() as demo:
    gr.Markdown("### Enter row index starting from 1")
    with gr.Row():
        with gr.Column():
            df_block = gr.Dataframe(df)
        with gr.Column():
            index_input = gr.Number(value=1, label="Row Index")
            audio_output = gr.Audio(label="Combined Speech")
            index_input.change(fn=generate_speech, inputs=index_input, outputs=audio_output)

# Launch the interface
demo.launch(debug=True)


In [ ]:
#@markdown [2] Gradio implementation: with Submit button
import gradio as gr
from gtts import gTTS
from pydub import AudioSegment
import pandas as pd
import tempfile
from io import BytesIO

url = "https://raw.githubusercontent.com/MK316/Spring2024/main/data/FKE.csv"
df = pd.read_csv(url)

# Function to generate speech and combine two audios
def generate_speech(index):
    # Adjust for zero-based index
    adjusted_index = int(index) - 1  # Ensure index is an integer

    # Check if the provided index is within the range of the DataFrame
    if adjusted_index < 0 or adjusted_index >= len(df):
        return "Index out of range", None

    # Get the Korean and English words from the DataFrame
    korean_word = df.iloc[adjusted_index]['Entry']
    english_word = df.iloc[adjusted_index]['Eword']

    # Generate TTS for Korean
    tts_ko = gTTS(text=korean_word, lang='ko')
    ko_buffer = BytesIO()
    tts_ko.write_to_fp(ko_buffer)
    ko_buffer.seek(0)
    ko_audio = AudioSegment.from_file(ko_buffer, format="mp3")

    # Generate TTS for English
    tts_en = gTTS(text=english_word, lang='en')
    en_buffer = BytesIO()
    tts_en.write_to_fp(en_buffer)
    en_buffer.seek(0)
    en_audio = AudioSegment.from_file(en_buffer, format="mp3")

    # Add a 2-second pause between the words
    pause = AudioSegment.silent(duration=700)

    # Combine the two audios with a pause in between
    combined_audio = ko_audio + pause + en_audio

    # Save combined audio to a temporary file
    with tempfile.NamedTemporaryFile(delete=False, suffix='.mp3') as fp:
        combined_audio.export(fp, format='mp3')
        # Return the file path
        return fp.name

# Use the Blocks API to create a more custom layout
with gr.Blocks() as demo:
    gr.Markdown("## Enter row number (SID) to generate speech for the Entry and English word.")
    with gr.Row():
        with gr.Column():
            df_block = gr.Dataframe(df)
        with gr.Column():
            index_input = gr.Number(value=1, label="Type the number to listen")
            submit_btn = gr.Button("Generate Audio")
            audio_output = gr.Audio(label="Audio: Korean vs. English")
            # Use the button's click event to trigger the generation
            submit_btn.click(fn=generate_speech, inputs=index_input, outputs=audio_output)

# Launch the interface
demo.launch(debug=True)
